<a href="https://colab.research.google.com/github/pseeej/MachineLearning_DCLAB/blob/master/DL%20for%20Everyone/22.%20Data%20Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, initializers, regularizers, metrics

In [ ]:
np.random.seed(3)
tf.random.set_seed(3)

In [ ]:
# data 수 늘리는 ImageDataGenerator
# 주어진 데이터를 이용해 변형된 이미지를 만들어 학습셋에 포함시키는 편리한 기능 제공.
# 근데 이거 아래처럼 인자 다 적용하면 불필요한 데이터 만들어내서 학습 시간 늘어나게 됨.
train_datagen = ImageDataGenerator(rescale=1./255,  # 주어진 이미지의 크기 바꿔줌. 정규화 과정.
                                   horizontal_flip = True,  # 수평 대칭 이미지를 50% 확률로 만들어 추가
                                   width_shift_range = 0.1, # 전체 크기의 10%
                                   height_shift_range = 0.1,  # 마찬가지로 위아래로 이동
                                   # rotation_range = 5,  # 정해진 각도만큼 이미지 회전
                                   # shear_range = 0.7, # 좌표 하나 고정시키고 다른 몇 개의 좌표 이동시키는 변환
                                   # zoom_range=[0.9, 2.2], # 정해진 범위 안에서 축소/확대
                                   # vertical_flip = True
                                   fill_mode = 'nearest') # 이미지를 축소/회전/이동할 때 생기는 빈 공간을 어떻게 채울지 결정. nearest는 가장 비슷한 색으로 채움.

In [ ]:
# 폴더에 저장된 데이터 불러오는 flow_from_directory
train_generator = train_datagen.flow_from_directory(
       './train',   #학습셋이 있는 폴더의 위치입니다.
       target_size=(150, 150),  # 이미지 크기
       batch_size=5,
       class_mode='binary') # 치매/정상 2진분류이므로 바이너리 모드로 실행

In [ ]:
# 테스트셋은 이미지 부풀리기 과정을 진행하지 않음
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    './test',  # 테스트셋이 있는 폴더 위치
    target_size = (150, 150),
    batch_size = 5,
    class_mode = 'binary')

In [ ]:
# 앞서 배운 CNN 모델을 만들어 적용하기
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

In [ ]:
# 모델 컴파일
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0002), metrics=['accuracy'])

In [ ]:
# 모델 실행
history = model.fit_generator(  
    train_generator,  # 학습모델
    steps_per_epoch = 100,  # 이미지 생성기에서 몇 개의 샘플 뽑을지 결정
    epochs=20,
    validation_data = test_generator, validation_steps=4  # 앞에서 만들어진 test_generator를 테스트셋으로 사용
)

In [ ]:
#결과를 그래프로 표현하는 부분입니다.
acc= history.history['accuracy']
val_acc= history.history['val_accuracy']
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))  
plt.plot(x_len, acc, marker='.', c="red", label='Trainset_acc')
plt.plot(x_len, val_acc, marker='.', c="lightcoral", label='Testset_acc')
plt.plot(x_len, y_vloss, marker='.', c="cornflowerblue", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

plt.legend(loc='upper right') 
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss/acc')
plt.show()